**We use ProsusAI/finbert model from the transformers library in order to perform sentiment analysis on the 8K text**

In [ ]:
!pip install transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 33.4 MB/s 
     |████████████████████████████████| 6.5 MB 20.9 MB/s 
     |████████████████████████████████| 895 kB 31.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
import torch
import pandas as pd
import json

In [ ]:
def findSentiment(sentence):
  input = tokenizer(sentence, padding = True, truncation = True,  return_tensors='pt')
  outputs = model(**input)
  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1) # [positive,negative,neutral]
  return prediction[0][0].item() , prediction[0][1].item() , prediction[0][2].item()


In [ ]:
json_file_name = "summarised_data.json"

In [ ]:
f = open(f'/content/{json_file_name}')
data = json.load(f)

years = [2020,2021]

companies = pd.read_csv("/content/GoodCom.csv")
ciks = companies["CIK"].astype(int).tolist()

cluster_features = [
['investors', 'stock', 'outstanding', 'demand', 'tribute'],
['acquire', 'secure', 'acquired', 'secured', 'agreement'], 
['revenue' ,'growth', 'profit', 'profitability', 'generating', 'generation', 'ebidta', 'earning', 'earnings', 'pay back','loss','debt'], 
['statement', 'statements', 'filed', 'report', 'reports', 'result', 'results'],
['dollar','hundred','thousand','million','billion','%','$'] ]

#We use the cluster features in order to cluster the sentences into 5 types of clusters
#Each cluster represents a particular type of financial statement

clustered_8k = dict()

for cik in ciks[0:3]:

    year_dict = dict()

    for year in years:

        try:
            cluster_dict = dict()
            for i in range(len(cluster_features)):
                cluster_dict[f"_{i}"] = []

            sentences = data[cik][year]
            
            for sentence_object in sentences:
                line = sentence_object['sentence']
                date = sentence_object['date']

                if ('common' in line) and ('stock' in line) and ('par' in line) and ('value' in line) and ('share' in line):
                    continue

                assigned = False
                for i in range(len(cluster_features)):
                    if assigned:
                        break
                    features = cluster_features[i]
                    for feature in features:
                        if feature in line:
                            line = line.capitalize()
                            positive , negative , neutral = findSentiment(line)
                            #For each sentence, we find its sentiment using the finbert model sentiment = ""
                            if positive>=0.7:
                              sentiment = "Positive"
                            elif negative>=0.7:
                              sentiment = "Negative"
                            else:
                              sentiment = "Neutral"
                            
                            cluster_dict[f"_{i}"].append({'sentence':line,'date':date,'sentiment':sentiment})
                            assigned = True
                            break

            year_dict[year] = cluster_dict
        except Exception as e:
            print(cik," ",year," ",e)
            continue

    clustered_8k[cik] = year_dict

    
with open(f"bulk_res.json", 'w') as f:
    json.dump(clustered_8k, f, indent=4)